In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
from datetime import datetime
import streamlit as st

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


In [ ]:
# Load the metadata
        df = pd.read_csv('metadata.csv')       
# Load the data
df = load_metadata()
       sample_data = {
        'cord_uid': [f'uid_{i}' for i in range(1000)],
        'title': [f'COVID-19 Research Paper {i} on various topics' for i in range(1000)],
        'abstract': [f'This is abstract {i} discussing COVID-19 pandemic impacts.' for i in range(1000)],
        'publish_time': np.random.choice(dates, 1000),
        'journal': np.random.choice(journals + [None] * 2, 1000),
        'authors': [f'Author {i}, Co-author {i}' for i in range(1000)],
        'url': [f'https://example.com/paper{i}' for i in range(1000)],
        'source_x': np.random.choice(['PubMed', 'PMC', 'WHO', 'CDC'], 1000)
    }
    df = pd.DataFrame(sample_data)

# Examine the data
print("=== First few rows ===")
print(df.head())

print("\n=== DataFrame info ===")
print(f"Dimensions: {df.shape[0]} rows, {df.shape[1]} columns")

print("\n=== Column names ===")
print(df.columns.tolist())

print("\n=== Data types ===")
print(df.dtypes)

print("\n=== Missing values ===")
missing_data = df.isnull().sum()
print(missing_data[missing_data > 0])

print("\n=== Basic statistics ===")
print(df.describe(include='all'))

In [6]:
def perform_analysis(df):
    """Perform comprehensive data analysis"""
    
    print("=== PERFORMING DATA ANALYSIS ===")
    
    # 1. Papers by publication year
    print("\n1. Papers by publication year:")
    yearly_counts = df['publication_year'].value_counts().sort_index()
    print(yearly_counts)
    
    # 2. Top journals
    print("\n2. Top journals publishing COVID-19 research:")
    top_journals = df['journal'].value_counts().head(10)
    print(top_journals)
    
    # 3. Most frequent words in titles
    print("\n3. Most frequent words in titles:")
    all_titles = ' '.join(df['title_clean'].dropna())
    words = re.findall(r'\b\w+\b', all_titles)
    
    # Remove common stop words
    stop_words = {'the', 'and', 'of', 'in', 'to', 'a', 'for', 'with', 'on', 'as', 'by', 
                 'an', 'at', 'from', 'that', 'is', 'are', 'this', 'these', 'those', 'or'}
    
    filtered_words = [word for word in words if word.lower() not in stop_words and len(word) > 2]
    word_freq = Counter(filtered_words).most_common(20)
    print("Top 20 words in titles:")
    for word, count in word_freq:
        print(f"{word}: {count}")
    
    return yearly_counts, top_journals, word_freq

def create_visualizations(df, yearly_counts, top_journals, word_freq):
    """Create comprehensive visualizations"""
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Publications over time
    axes[0, 0].plot(yearly_counts.index, yearly_counts.values, marker='o', linewidth=2)
    axes[0, 0].set_title('COVID-19 Publications Over Time', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Number of Publications')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Top journals
    top_10_journals = top_journals.head(10)
    axes[0, 1].barh(range(len(top_10_journals)), top_10_journals.values)
    axes[0, 1].set_yticks(range(len(top_10_journals)))
    axes[0, 1].set_yticklabels(top_10_journals.index)
    axes[0, 1].set_title('Top 10 Journals by Publication Count', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Number of Publications')
    
    # 3. Word cloud
    all_titles = ' '.join(df['title_clean'].dropna())
    wordcloud = WordCloud(width=800, height=400, background_color='white', 
                         max_words=100).generate(all_titles)
    axes[1, 0].imshow(wordcloud, interpolation='bilinear')
    axes[1, 0].set_title('Word Cloud of Paper Titles', fontsize=14, fontweight='bold')
    axes[1, 0].axis('off')
    
    # 4. Distribution of papers by source (if available)
    if 'source_x' in df.columns:
        source_counts = df['source_x'].value_counts().head(8)
        axes[1, 1].pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%')
        axes[1, 1].set_title('Paper Distribution by Source', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('cord19_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Additional visualization: Abstract word count distribution
    plt.figure(figsize=(10, 6))
    plt.hist(df[df['abstract_word_count'] > 0]['abstract_word_count'], bins=50, alpha=0.7)
    plt.title('Distribution of Abstract Word Count', fontsize=14, fontweight='bold')
    plt.xlabel('Word Count')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()



In [ ]:
# Save the Streamlit app as a separate file: app.py

"""
# CORD-19 Data Explorer App (app.py)

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import numpy as np

# Set page configuration
st.set_page_config(
    page_title="CORD-19 Data Explorer",
    page_icon="📊",
    layout="wide"
)

# Title and description
st.title("CORD-19 COVID-19 Research Data Explorer")
st.write("""
This interactive dashboard explores the COVID-19 Open Research Dataset (CORD-19), 
containing scientific papers about COVID-19 and related coronaviruses.
""")

# Sidebar for controls
st.sidebar.header("Controls")
st.sidebar.write("Adjust the parameters to explore the data:")

# Load data (in a real app, you'd load your actual cleaned data)
@st.cache_data
def load_data():
    # This would load your actual cleaned DataFrame
    # For demo purposes, we'll use the sample data creation logic
    return df_clean

df = load_data()

# Year range selector
if 'publication_year' in df.columns:
    min_year = int(df['publication_year'].min())
    max_year = int(df['publication_year'].max())
    
    year_range = st.sidebar.slider(
        "Select Year Range",
        min_value=min_year,
        max_value=max_year,
        value=(min_year, max_year)
    )
    
    # Filter data based on selection
    filtered_df = df[
        (df['publication_year'] >= year_range[0]) & 
        (df['publication_year'] <= year_range[1])
    ]
else:
    filtered_df = df

# Journal selector
if 'journal' in df.columns:
    journals = ['All'] + sorted(df['journal'].unique().tolist())
    selected_journal = st.sidebar.selectbox("Select Journal", journals)
    
    if selected_journal != 'All':
        filtered_df = filtered_df[filtered_df['journal'] == selected_journal]

# Main content area
col1, col2 = st.columns([2, 1])

with col1:
    st.subheader("Key Metrics")
    
    # Display metrics
    metric_col1, metric_col2, metric_col3, metric_col4 = st.columns(4)
    
    with metric_col1:
        st.metric("Total Papers", len(filtered_df))
    
    with metric_col2:
        if 'publication_year' in filtered_df.columns:
            st.metric("Latest Year", int(filtered_df['publication_year'].max()))
    
    with metric_col3:
        if 'has_abstract' in filtered_df.columns:
            abstracts_available = filtered_df['has_abstract'].sum()
            st.metric("Abstracts Available", abstracts_available)
    
    with metric_col4:
        if 'journal' in filtered_df.columns:
            unique_journals = filtered_df['journal'].nunique()
            st.metric("Unique Journals", unique_journals)

# Visualizations
st.subheader("Data Visualizations")

# Create tabs for different visualizations
tab1, tab2, tab3, tab4 = st.tabs([
    "Publications Over Time", 
    "Top Journals", 
    "Title Word Cloud",
    "Data Sample"
])

with tab1:
    st.write("### COVID-19 Publications Over Time")
    if 'publication_year' in filtered_df.columns:
        yearly_counts = filtered_df['publication_year'].value_counts().sort_index()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(yearly_counts.index, yearly_counts.values, marker='o', linewidth=2)
        ax.set_xlabel('Year')
        ax.set_ylabel('Number of Publications')
        ax.grid(True, alpha=0.3)
        ax.set_title('Publications Over Time')
        st.pyplot(fig)
    else:
        st.write("Publication year data not available.")

with tab2:
    st.write("### Top Journals")
    if 'journal' in filtered_df.columns:
        top_journals = filtered_df['journal'].value_counts().head(10)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.barh(range(len(top_journals)), top_journals.values)
        ax.set_yticks(range(len(top_journals)))
        ax.set_yticklabels(top_journals.index)
        ax.set_xlabel('Number of Publications')
        ax.set_title('Top 10 Journals')
        st.pyplot(fig)
    else:
        st.write("Journal data not available.")

with tab3:
    st.write("### Word Cloud of Paper Titles")
    if 'title_clean' in filtered_df.columns:
        all_titles = ' '.join(filtered_df['title_clean'].dropna())
        
        if all_titles.strip():
            wordcloud = WordCloud(width=800, height=400, background_color='white', 
                                max_words=100).generate(all_titles)
            
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.imshow(wordcloud, interpolation='bilinear')
            ax.axis('off')
            ax.set_title('Word Cloud of Paper Titles')
            st.pyplot(fig)
        else:
            st.write("No title data available for word cloud.")
    else:
        st.write("Title data not available.")

with tab4:
    st.write("### Sample Data")
    st.dataframe(filtered_df.head(100))

# Additional information
st.sidebar.subheader("ℹ️ About")
st.sidebar.write("""
This app analyzes the CORD-19 dataset containing COVID-19 research papers.

**Data Source:** COVID-19 Open Research Dataset (CORD-19)
""")


In [8]:
# documentation_report.md

"""
# CORD-19 Data Analysis Project - Documentation and Reflection

## Project Overview
This project analyzes the CORD-19 dataset, which contains metadata about COVID-19 research papers. The analysis includes data loading, cleaning, exploration, visualization, and an interactive Streamlit application.

## Key Findings

### 1. Data Characteristics
- **Dataset Size**: [Number] rows, [Number] columns
- **Time Period**: Papers published from [Start Year] to [End Year]
- **Key Columns**: Title, Abstract, Publication Date, Journal, Authors

### 2. Publication Trends
- **Peak Publication Year**: [Year] with [Number] papers
- **Growth Pattern**: [Description of publication growth over time]

### 3. Journal Distribution
- **Top Journals**: [List top 3-5 journals]
- **Publication Concentration**: [Description of how publications are distributed across journals]

### 4. Content Analysis
- **Common Themes**: [Key topics from title word frequency]
- **Abstract Availability**: [Percentage] of papers have abstracts

## Technical Implementation

### Data Cleaning Challenges
1. **Missing Values**: Handled missing titles, abstracts, and publication dates appropriately
2. **Data Consistency**: Standardized journal names and publication formats
3. **Text Processing**: Cleaned and normalized title text for analysis

### Analysis Techniques
1. **Time Series Analysis**: Tracked publication patterns over time
2. **Text Mining**: Extracted key themes from paper titles
3. **Comparative Analysis**: Compared publication output across different sources

### Visualization Approach
- Used multiple chart types (line, bar, word cloud) for comprehensive insights
- Implemented interactive filters in Streamlit for user exploration
- Ensured visualizations are clear and interpretable

## Challenges Faced

1. **Data Quality**: Inconsistent formatting in source data required careful cleaning
2. **Scale Management**: Large dataset required efficient processing techniques
3. **Visualization Selection**: Choosing the most effective charts for different data types

## Lessons Learned

1. **Data Preparation**: 80% of the work is in cleaning and preparing data
2. **Iterative Analysis**: Multiple passes through data reveal different insights
3. **User Experience**: Interactive tools significantly enhance data exploration

## Future Enhancements

1. **Advanced NLP**: Implement topic modeling on abstracts
2. **Citation Analysis**: Include citation networks if available
3. **Real-time Updates**: Connect to live data source
4. **Collaboration Analysis**: Map author collaboration networks

## Code Quality Features

- **Modular Design**: Separate functions for loading, cleaning, analysis
- **Comprehensive Documentation**: Clear comments and docstrings
- **Error Handling**: Robust handling of edge cases
- **Reproducibility**: Seed values for random operations

## Conclusion
This project successfully demonstrates a complete data analysis pipeline from raw data to interactive insights. The CORD-19 dataset provides valuable insights into the rapid research response to the COVID-19 pandemic.
"""

# Reflection on the project
def project_reflection():
    """Reflect on the project challenges and learnings"""
    
    reflection = """
    PROJECT REFLECTION:
    
    SUCCESSES:
    1. Complete pipeline implementation from data loading to interactive app
    2. Comprehensive data cleaning and preparation
    3. Multiple visualization types for different insights
    4. User-friendly Streamlit interface
    
    CHALLENGES:
    1. Handling large dataset efficiently
    2. Dealing with inconsistent data quality
    3. Choosing appropriate visualizations
    4. Balancing complexity with usability
    
    KEY LEARNINGS:
    1. Importance of thorough data exploration before analysis
    2. Value of interactive tools for data discovery
    3. Need for flexible data cleaning approaches
    4. Benefits of modular code organization
    
    AREAS FOR IMPROVEMENT:
    1. Add more advanced text analysis (sentiment, topics)
    2. Implement performance optimizations for larger datasets
    3. Add more interactive filtering options
    4. Include statistical testing for observed patterns
    """
    
    print(reflection)

project_reflection(


    PROJECT REFLECTION:
    
    SUCCESSES:
    1. Complete pipeline implementation from data loading to interactive app
    2. Comprehensive data cleaning and preparation
    3. Multiple visualization types for different insights
    4. User-friendly Streamlit interface
    
    CHALLENGES:
    1. Handling large dataset efficiently
    2. Dealing with inconsistent data quality
    3. Choosing appropriate visualizations
    4. Balancing complexity with usability
    
    KEY LEARNINGS:
    1. Importance of thorough data exploration before analysis
    2. Value of interactive tools for data discovery
    3. Need for flexible data cleaning approaches
    4. Benefits of modular code organization
    
    AREAS FOR IMPROVEMENT:
    1. Add more advanced text analysis (sentiment, topics)
    2. Implement performance optimizations for larger datasets
    3. Add more interactive filtering options
    4. Include statistical testing for observed patterns
    
